In [ ]:
# 查看当前挂载的数据集目录, 该目录下的变更重启环境后会自动还原
# View dataset directory. This directory will be recovered automatically after resetting environment. 
!ls /home/aistudio/data

In [0]:
# 查看工作区文件, 该目录下的变更将会持久保存. 请及时清理不必要的文件, 避免加载过慢.
# View personal work directory. All changes under this directory will be kept even after reset. Please clean unnecessary files in time to speed up environment loading.
!ls /home/aistudio/work

In [ ]:
# 如果需要进行持久化安装, 需要使用持久化路径, 如下方代码示例:
# If a persistence installation is required, you need to use the persistence path as the following:
!mkdir /home/aistudio/external-libraries
!pip install beautifulsoup4 -t /home/aistudio/external-libraries

In [ ]:
# 同时添加如下代码, 这样每次环境(kernel)启动的时候只要运行下方代码即可:
# Also add the following code, so that every time the environment (kernel) starts, just run the following code:
import sys
sys.path.append('/home/aistudio/external-libraries')

In [1]:
!git clone https://github.com/PaddlePaddle/PaddleOCR.git

Cloning into 'PaddleOCR'...
remote: Enumerating objects: 261, done.
remote: Counting objects: 100% (261/261), done.
remote: Compressing objects: 100% (201/201), done.
remote: Total 1976 (delta 142), reused 131 (delta 60), pack-reused 1715
Receiving objects: 100% (1976/1976), 26.56 MiB | 27.00 KiB/s, done.
Resolving deltas: 100% (1195/1195), done.
Checking connectivity... done.


In [1]:
%cd PaddleOCR/

/home/aistudio/PaddleOCR


In [2]:
# 安装依赖库
!pip install -r requirments.txt -i https://mirror.baidu.com/pypi/simple

Looking in indexes: https://mirror.baidu.com/pypi/simple
     |████████████████████████████████| 1.8MB 14.9MB/s eta 0:00:01
     |████████████████████████████████| 952kB 15.7MB/s eta 0:00:01
     |████████████████████████████████| 133kB 22.5MB/s eta 0:00:01
     |████████████████████████████████| 870kB 12.7MB/s eta 0:00:01
     |████████████████████████████████| 12.5MB 8.2MB/s eta 0:00:011
     |████████████████████████████████| 143kB 65.1MB/s eta 0:00:01
     |████████████████████████████████| 4.4MB 28.5MB/s eta 0:00:01
  Created wheel for lmdb: filename=lmdb-0.98-cp37-cp37m-linux_x86_64.whl size=264310 sha256=8b5e433ebb96b0504d7122d5a2a1dd731eb0fb6e077f2603881640b58d886a19
  Stored in directory: /home/aistudio/.cache/pip/wheels/53/a0/7a/eb9ec000443fad79caaa8d70da9a40a54a313c9bb5f682fa36
Successfully built lmdb


## 2. 训练文字检测模型

### 2.1. 数据准备

In [4]:
!mkdir train_data

In [6]:
!ls

configs  LICENSE  README_en.md	requirments.txt  train_data
doc	 ppocr	  README.md	tools


In [7]:
!cd train_data/ && unzip -o images.zip

Archive:  images.zip
   creating: images/
   creating: images/test/
  inflating: images/test.txt         
  inflating: images/test/20200521105032.png  
  inflating: images/test/20200521105043.png  
  inflating: images/test/20200525142637.png  
  inflating: images/test/20200525142718.png  
  inflating: images/test/20200525142733.png  
  inflating: images/test/20200525142753.png  
  inflating: images/test/20200601092642.png  
  inflating: images/test/20200601092726.png  
  inflating: images/test/20200601092807.png  
  inflating: images/test/20200601092846.png  
  inflating: images/test/20200601092857.png  
   creating: images/train/
  inflating: images/train.txt        
  inflating: images/train/20200521104812.png  
  inflating: images/train/20200521104904.png  
  inflating: images/train/20200521104928.png  
  inflating: images/train/20200521104957.png  
  inflating: images/train/20200521105007.png  
  inflating: images/train/20200521105020.png  
  inflating: images/train/20200525090902.

提供的标注文件格式为：
```
" 图像文件名                    json.dumps编码的图像标注信息"
test/img_61.jpg    [{"transcription": "MASA", "points": [[310, 104], [416, 141], [418, 216], [312, 179]], ...}]
```
json.dumps编码前的图像标注信息是包含多个字典的list，字典中的$points$表示文本框的四个点的坐标(x, y)，从左上角的点开始顺时针排列。
$transcription$表示当前文本框的文字，在文本检测任务中并不需要这个信息。
如果您想在其他数据集上训练PaddleOCR，可以按照上述形式构建标注文件。


### 2.2 快速启动训练

首先下载pretrain model，PaddleOCR的检测模型目前支持两种backbone，分别是MobileNetV3、ResNet50_vd，
您可以根据需求使用[PaddleClas](https://github.com/PaddlePaddle/PaddleClas/tree/master/ppcls/modeling/architectures)中的模型更换backbone。

In [8]:
# 下载MobileNetV3的预训练模型
!wget -P ./pretrain_models/ https://paddle-imagenet-models-name.bj.bcebos.com/MobileNetV3_large_x0_5_pretrained.tar
! cd pretrain_models/ && tar xf MobileNetV3_large_x0_5_pretrained.tar
# 下载ResNet50的预训练模型
!wget -P ./pretrain_models/ https://paddle-imagenet-models-name.bj.bcebos.com/ResNet50_vd_ssld_pretrained.tar
! cd pretrain_models/ && tar xf ResNet50_vd_ssld_pretrained.tar

--2020-06-23 15:31:11--  https://paddle-imagenet-models-name.bj.bcebos.com/MobileNetV3_large_x0_5_pretrained.tar
Resolving paddle-imagenet-models-name.bj.bcebos.com (paddle-imagenet-models-name.bj.bcebos.com)... 182.61.200.195, 182.61.200.229
Connecting to paddle-imagenet-models-name.bj.bcebos.com (paddle-imagenet-models-name.bj.bcebos.com)|182.61.200.195|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9974980 (9.5M) [application/x-tar]
Saving to: ‘./pretrain_models/MobileNetV3_large_x0_5_pretrained.tar’

MobileNetV3_large_x 100%[===================>]   9.51M  10.5MB/s    in 0.9s    

2020-06-23 15:31:12 (10.5 MB/s) - ‘./pretrain_models/MobileNetV3_large_x0_5_pretrained.tar’ saved [9974980/9974980]

--2020-06-23 15:31:13--  https://paddle-imagenet-models-name.bj.bcebos.com/ResNet50_vd_ssld_pretrained.tar
Resolving paddle-imagenet-models-name.bj.bcebos.com (paddle-imagenet-models-name.bj.bcebos.com)... 182.61.200.229, 182.61.200.195
Connecting to paddle-imagene

In [13]:
# 训练backbone为MobileNetV3的db算法的检测模型
!export PYTHONPATH=. && python3 tools/train.py -c configs/det/det_mv3_db_meter.yml 


上述指令中，通过-c 选择训练使用configs/det/det_db_mv3.yml配置文件。
有关配置文件的详细解释，请参考/doc目录下的config.md。

您也可以通过-o参数在不需要修改yml文件的情况下，改变训练的参数，比如，调整训练的学习率为0.0001
```
python3 tools/train.py -c configs/det/det_db_mv3.yml -o Optimizer.base_lr=0.0001
```


### 2.3. 测试检测效果

训练过程中的检测模型保存在'./output/det_db/'中，模型保存的位置通过yml配置文件的Global.save_model_dir参数设置。

使用训练好的模型测试单张图像的检测效果。

In [14]:
!export PYTHONPATH=. && python3 tools/infer_det.py -c configs/det/det_mv3_db_meter.yml -o TestReader.infer_img="./train_data/images/test/"  Global.checkpoints="./output/det_db/best_accuracy.pdparams"

{'Global': {'algorithm': 'DB', 'use_gpu': True, 'epoch_num': 300, 'log_smooth_window': 20, 'print_batch_step': 2, 'save_model_dir': './output/det_db/', 'save_epoch_step': 50, 'eval_batch_step': 100, 'train_batch_size_per_card': 16, 'test_batch_size_per_card': 1, 'image_shape': [3, 640, 640], 'reader_yml': './configs/det/det_db_meter_reader.yml', 'pretrain_weights': './pretrain_models/MobileNetV3_large_x0_5_pretrained/', 'checkpoints': './output/det_db/best_accuracy.pdparams', 'save_res_path': './output/det_db/predicts_db.txt', 'save_inference_dir': None}, 'Architecture': {'function': 'ppocr.modeling.architectures.det_model,DetModel'}, 'Backbone': {'function': 'ppocr.modeling.backbones.det_mobilenet_v3,MobileNetV3', 'scale': 0.5, 'model_name': 'large'}, 'Head': {'function': 'ppocr.modeling.heads.det_db_head,DBHead', 'model_name': 'large', 'k': 50, 'inner_channels': 96, 'out_channels': 2}, 'Loss': {'function': 'ppocr.modeling.losses.det_db_loss,DBLoss', 'balance_loss': True, 'main_loss_t

## 将训练好的文字识别模型做剪裁，制作数据集供文字识别模型使用

### 2.4. 首先需要将预训练的模型转Inference模型

#### 以MobileNetV3为backbone训练的DB算法，将训练好的模型转换成inference模型只需要运行如下命令：

In [3]:
!python3 tools/export_model.py -c configs/det/det_mv3_db_meter.yml -o Global.checkpoints=./output/det_db/best_accuracy Global.save_inference_dir=./inference/det_db/

2020-06-24 09:15:02,967-INFO: {'Global': {'algorithm': 'DB', 'use_gpu': True, 'epoch_num': 300, 'log_smooth_window': 20, 'print_batch_step': 2, 'save_model_dir': './output/det_db/', 'save_epoch_step': 50, 'eval_batch_step': 100, 'train_batch_size_per_card': 16, 'test_batch_size_per_card': 1, 'image_shape': [3, 640, 640], 'reader_yml': './configs/det/det_db_meter_reader.yml', 'pretrain_weights': './pretrain_models/MobileNetV3_large_x0_5_pretrained/', 'checkpoints': './output/det_db/best_accuracy', 'save_res_path': './output/det_db/predicts_db.txt', 'save_inference_dir': './inference/det_db/'}, 'Architecture': {'function': 'ppocr.modeling.architectures.det_model,DetModel'}, 'Backbone': {'function': 'ppocr.modeling.backbones.det_mobilenet_v3,MobileNetV3', 'scale': 0.5, 'model_name': 'large'}, 'Head': {'function': 'ppocr.modeling.heads.det_db_head,DBHead', 'model_name': 'large', 'k': 50, 'inner_channels': 96, 'out_channels': 2}, 'Loss': {'function': 'ppocr.modeling.losses.det_db_loss,DBLos

#### 接下来要开始生成文字识别的数据集

In [21]:
!export PYTHONPATH=. && python3 tools/infer/predict_system_export_data.py --image_dir="./train_data/images/test/" --det_model_dir="./inference/det_db/" 

20200525142733
Predict time of ./train_data/images/test/20200525142733.png: 1.873s
20200601092642
Predict time of ./train_data/images/test/20200601092642.png: 0.047s
20200525142637
Predict time of ./train_data/images/test/20200525142637.png: 0.049s
20200521105032
Predict time of ./train_data/images/test/20200521105032.png: 0.034s
20200601092857
Predict time of ./train_data/images/test/20200601092857.png: 0.041s
20200525142753
Predict time of ./train_data/images/test/20200525142753.png: 0.026s
20200521105043
Predict time of ./train_data/images/test/20200521105043.png: 0.031s
20200601092726
Predict time of ./train_data/images/test/20200601092726.png: 0.042s
20200601092807
Predict time of ./train_data/images/test/20200601092807.png: 0.042s
20200601092846
Predict time of ./train_data/images/test/20200601092846.png: 0.040s
20200525142718
Predict time of ./train_data/images/test/20200525142718.png: 0.040s


## 3. 训练文字识别模型


### 3.1. 数据准备

In [22]:
!ls

configs    LICENSE	 ppocr		  README.md	   train_data
doc	   output	 pretrain_models  requirments.txt
inference  output_meter  README_en.md	  tools


若您本地没有数据集，可以在官网下载 [icdar2015](http://rrc.cvc.uab.es/?ch=4&com=downloads) 数据，用于快速验证。也可以参考[DTRB](https://github.com/clovaai/deep-text-recognition-benchmark#download-lmdb-dataset-for-traininig-and-evaluation-from-here)，下载 benchmark 所需的lmdb格式数据集。

* 使用自己数据集：

若您希望使用自己的数据进行训练，请参考下文组织您的数据。

- 训练集

首先请将训练图片放入同一个文件夹（train_images），并用一个txt文件（rec_gt_train.txt）记录图片路径和标签。

* 注意： 默认请将图片路径和图片标签用 \t 分割，如用其他方式分割将造成训练报错

```
" 图像文件名                 图像标注信息 "

train_data/train_0001.jpg   简单可依赖
train_data/train_0002.jpg   用科技让复杂的世界更简单
```
最终训练集应有如下文件结构：

```
|-train_data
    |-ic15_data
        |- rec_gt_train.txt
        |- train
            |- word_001.png
            |- word_002.jpg
            |- word_003.jpg
            | ...
```

- 测试集

同训练集类似，测试集也需要提供一个包含所有图片的文件夹（test）和一个rec_gt_test.txt，测试集的结构如下所示：

```
|-train_data
    |-ic15_data
        |- rec_gt_test.txt
        |- test
            |- word_001.jpg
            |- word_002.jpg
            |- word_003.jpg
            | ...
```

In [24]:
! cd train_data/ && unzip -o output_meter.zip

Archive:  output_meter.zip
   creating: output_meter/
   creating: output_meter/test/
  inflating: output_meter/test.txt   
  inflating: output_meter/test/img_crop_20200521104928.jpg  
  inflating: output_meter/test/img_crop_20200521105007.jpg  
  inflating: output_meter/test/img_crop_20200525142551.jpg  
  inflating: output_meter/test/img_crop_20200525142612.jpg  
  inflating: output_meter/test/img_crop_20200525142637.jpg  
  inflating: output_meter/test/img_crop_20200529093403.jpg  
  inflating: output_meter/test/img_crop_20200601092846.jpg  
  inflating: output_meter/test/img_crop_20200601092857.jpg  
  inflating: output_meter/test/img_crop_20200601092919.jpg  
   creating: output_meter/train/
  inflating: output_meter/train.txt  
  inflating: output_meter/train/img_crop_20200521104812.jpg  
  inflating: output_meter/train/img_crop_20200521104904.jpg  
  inflating: output_meter/train/img_crop_20200521105020.jpg  
  inflating: output_meter/train/img_crop_20200521105032.jpg  
  inflat

### 3.2. 快速启动训练

首先下载pretrain model，PaddleOCR的检测模型目前支持两种backbone，分别是MobileNetV3、ResNet50_vd，
您可以根据需求使用[PaddleClas](https://github.com/PaddlePaddle/PaddleClas/tree/master/ppcls/modeling/architectures)中的模型更换backbone。
本节将以 CRNN 识别模型为例：

首先下载pretrain model，您可以下载训练好的模型在 icdar2015 数据上进行finetune。

In [25]:
!wget -P ./pretrain_models/ https://paddleocr.bj.bcebos.com/rec_mv3_none_bilstm_ctc.tar
!cd pretrain_models && tar -xf rec_mv3_none_bilstm_ctc.tar && rm -rf rec_mv3_none_bilstm_ctc.tar

--2020-06-24 15:33:37--  https://paddleocr.bj.bcebos.com/rec_mv3_none_bilstm_ctc.tar
Resolving paddleocr.bj.bcebos.com (paddleocr.bj.bcebos.com)... 182.61.200.229, 182.61.200.195
Connecting to paddleocr.bj.bcebos.com (paddleocr.bj.bcebos.com)|182.61.200.229|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 25964544 (25M) [application/x-tar]
Saving to: ‘./pretrain_models/rec_mv3_none_bilstm_ctc.tar’

rec_mv3_none_bilstm 100%[===================>]  24.76M  7.40MB/s    in 4.0s    

2020-06-24 15:33:41 (6.15 MB/s) - ‘./pretrain_models/rec_mv3_none_bilstm_ctc.tar’ saved [25964544/25964544]



### 开始训练

In [26]:
!export PYTHONPATH=. && python3 tools/train.py -c configs/rec/rec_meter_train.yml

2020-06-24 15:38:22,241-INFO: {'Global': {'algorithm': 'CRNN', 'use_gpu': True, 'epoch_num': 300, 'log_smooth_window': 20, 'print_batch_step': 10, 'save_model_dir': './output/rec_CRNN', 'save_epoch_step': 50, 'eval_batch_step': 100, 'train_batch_size_per_card': 16, 'test_batch_size_per_card': 1, 'image_shape': [3, 32, 100], 'max_text_length': 25, 'character_type': 'en', 'loss_type': 'ctc', 'reader_yml': './configs/rec/rec_meter_reader.yml', 'pretrain_weights': './pretrain_models/rec_mv3_none_bilstm_ctc/best_accuracy', 'checkpoints': None, 'save_inference_dir': None, 'infer_img': None}, 'Architecture': {'function': 'ppocr.modeling.architectures.rec_model,RecModel'}, 'Backbone': {'function': 'ppocr.modeling.backbones.rec_mobilenet_v3,MobileNetV3', 'scale': 0.5, 'model_name': 'large'}, 'Head': {'function': 'ppocr.modeling.heads.rec_ctc_head,CTCPredict', 'encoder_type': 'rnn', 'SeqRNN': {'hidden_size': 96}}, 'Loss': {'function': 'ppocr.modeling.losses.rec_ctc_loss,CTCLoss'}, 'Optimizer': {

### 3.3 测试识别效果

测试单张图像的识别结果

In [33]:
!export PYTHONPATH=. && python3 tools/infer_rec.py -c configs/rec/rec_meter_train.yml -o Global.infer_img="./train_data/output_meter/test/img_crop_20200521104928.jpg"  Global.checkpoints="./output/rec_CRNN/best_accuracy.pdparams"

2020-06-24 16:00:20,943-INFO: {'Global': {'algorithm': 'CRNN', 'use_gpu': True, 'epoch_num': 300, 'log_smooth_window': 20, 'print_batch_step': 10, 'save_model_dir': './output/rec_CRNN', 'save_epoch_step': 50, 'eval_batch_step': 100, 'train_batch_size_per_card': 16, 'test_batch_size_per_card': 1, 'image_shape': [3, 32, 100], 'max_text_length': 25, 'character_type': 'en', 'loss_type': 'ctc', 'reader_yml': './configs/rec/rec_meter_reader.yml', 'pretrain_weights': './pretrain_models/rec_mv3_none_bilstm_ctc/best_accuracy', 'checkpoints': './output/rec_CRNN/best_accuracy.pdparams', 'save_inference_dir': None, 'infer_img': './train_data/output_meter/test/img_crop_20200521104928.jpg'}, 'Architecture': {'function': 'ppocr.modeling.architectures.rec_model,RecModel'}, 'Backbone': {'function': 'ppocr.modeling.backbones.rec_mobilenet_v3,MobileNetV3', 'scale': 0.5, 'model_name': 'large'}, 'Head': {'function': 'ppocr.modeling.heads.rec_ctc_head,CTCPredict', 'encoder_type': 'rnn', 'SeqRNN': {'hidden_s

## 将预训练的模型转Inference模型

In [34]:
#模型推理
!export PYTHONPATH=. && python3 tools/export_model.py -c configs/rec/rec_meter_train.yml -o Global.checkpoints="./output/rec_CRNN/best_accuracy"  Global.save_inference_dir="./inference/rec"

2020-06-24 16:03:52,223-INFO: {'Global': {'algorithm': 'CRNN', 'use_gpu': True, 'epoch_num': 300, 'log_smooth_window': 20, 'print_batch_step': 10, 'save_model_dir': './output/rec_CRNN', 'save_epoch_step': 50, 'eval_batch_step': 100, 'train_batch_size_per_card': 16, 'test_batch_size_per_card': 1, 'image_shape': [3, 32, 100], 'max_text_length': 25, 'character_type': 'en', 'loss_type': 'ctc', 'reader_yml': './configs/rec/rec_meter_reader.yml', 'pretrain_weights': './pretrain_models/rec_mv3_none_bilstm_ctc/best_accuracy', 'checkpoints': './output/rec_CRNN/best_accuracy', 'save_inference_dir': './inference/rec', 'infer_img': None}, 'Architecture': {'function': 'ppocr.modeling.architectures.rec_model,RecModel'}, 'Backbone': {'function': 'ppocr.modeling.backbones.rec_mobilenet_v3,MobileNetV3', 'scale': 0.5, 'model_name': 'large'}, 'Head': {'function': 'ppocr.modeling.heads.rec_ctc_head,CTCPredict', 'encoder_type': 'rnn', 'SeqRNN': {'hidden_size': 96}}, 'Loss': {'function': 'ppocr.modeling.los

## 检测推理模型,最后一哆嗦了

#### 这里面出现了一个问题，就是字典需要修改 具体需要参考：https://github.com/PaddlePaddle/PaddleOCR/blob/develop/doc/doc_ch/inference.md

In [40]:
# 快速运行
!export PYTHONPATH=. && python3 tools/infer/predict_system.py --image_dir="./train_data/images/train/20200529093420.png" --det_model_dir="./inference/det_db/"  --rec_model_dir="./inference/rec/" --rec_char_dict_path="./ppocr/utils/meter_dict.txt"

Predict time of ./train_data/images/train/20200529093420.png: 1.864s
00001, 0.998
The visualized image saved in ./inference_results/20200529093420.png


In [1]:
!ls

587961.ipynb  data  PaddleOCR  work


# 如何从Paddle保存的预测模型转为PaddleHub格式可部署的模型

请点击[此处](https://ai.baidu.com/docs#/AIStudio_Project_Notebook/a38e5576)查看本环境基本用法.  <br>
Please click [here ](https://ai.baidu.com/docs#/AIStudio_Project_Notebook/a38e5576) for more detailed instructions. 